In [1]:
import pandas as pd
import seaborn as sns

%load_ext google.cloud.bigquery

# Tenure Paid Lens

In [2]:
%%bigquery --project nbcu-ds-sandbox-a-001
SELECT  tenure_paid_lens
       ,CASE WHEN first_paying_date is null THEN 'Never Paid'  ELSE 'Prev Paid' END AS never_paid
       ,CASE WHEN first_premium_date is null THEN 'Never Premium'  ELSE 'Prev Premium' END AS never_premium
       ,COUNT(*)
FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
WHERE report_date = '2023-01-31'
GROUP BY  1,2,3
order by 1,2,3

Query is running:   0%|          |

Downloading:   0%|          |

,tenure_paid_lens,never_paid,never_premium,f0_
0,0-90 days,Never Paid,Never Premium,2254659
1,0-90 days,Never Paid,Prev Premium,476605
2,0-90 days,Prev Paid,Never Premium,1183021
3,0-90 days,Prev Paid,Prev Premium,4678693
4,91+ days,Never Paid,Never Premium,36021227
5,91+ days,Never Paid,Prev Premium,18660741
6,91+ days,Prev Paid,Never Premium,4731209
7,91+ days,Prev Paid,Prev Premium,23299062
8,N/A,Never Paid,Never Premium,19909351
9,N/A,Never Paid,Prev Premium,405118


In [6]:
%%bigquery --project nbcu-ds-sandbox-a-001
SELECT  tenure_paid_lens
       ,CASE WHEN first_premium_date is null THEN 'Never Premium'  ELSE 'Prev Premium' END AS never_premium
       ,CASE WHEN first_paying_date is null THEN 'Never Paid'  ELSE 'Prev Paid' END AS never_paid
       ,CASE WHEN date_of_last_view is null THEN 'Yes - Never Watched'  ELSE 'No - Watched' END AS abandoned_maa
       ,CASE WHEN u.gross_add_date is null THEN 'Never MAA'  ELSE 'Prev MAA' END AS user_gross_add_date_missing
       ,CASE WHEN d.gross_add_date is null THEN 'Never MAA'  ELSE 'Prev MAA' END AS device_gross_add_date_missing
       ,COUNT(u.adobe_tracking_id) as users
FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` u
LEFT JOIN `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES` d
ON u.adobe_tracking_id = d.adobe_tracking_id
WHERE u.report_date = '2023-01-31' AND d.report_date = '2023-01-31'
and tenure_paid_lens = 'N/A'
GROUP BY 1,2,3,4,5,6
order by 1,4,2,3,5,6

Query is running:   0%|          |

Downloading:   0%|          |

,tenure_paid_lens,never_premium,never_paid,abandoned_maa,user_gross_add_date_missing,device_gross_add_date_missing,users
0,N/A,Never Premium,Never Paid,No - Watched,Never MAA,Never MAA,918358
1,N/A,Prev Premium,Never Paid,No - Watched,Never MAA,Never MAA,12553
2,N/A,Never Premium,Never Paid,Yes - Never Watched,Never MAA,Never MAA,18990993
3,N/A,Prev Premium,Never Paid,Yes - Never Watched,Never MAA,Never MAA,392565


MAA is defined as:
   - Any Premium+ Account
   - Premium accounts managed directly by NBCU (and are not managed through a partnership or MVPD)
   - Any Accounts currently on Premium or Premium+ trial
   - Non-paying accounts (free entitlement or Premium accounts from a partner/MVPD) that have a content start in the last 30 days."

No gross add date when bundled premium or free

In [20]:
%%bigquery --project nbcu-ds-sandbox-a-001
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.tenure_paid_lens_analysis` AS 

WITH cte AS
(
	SELECT  u.adobe_tracking_id
	       ,tenure_paid_lens
	       ,CASE WHEN first_premium_date is null THEN 'Never Premium'  ELSE 'Prev Premium' END AS never_premium
	       ,CASE WHEN first_paying_date is null THEN 'Never Paid'  ELSE 'Prev Paid' END        AS never_paid
	FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` u
	LEFT JOIN `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES` d
	ON u.adobe_tracking_id = d.adobe_tracking_id
	WHERE u.report_date = '2023-01-31'
	AND d.report_date = '2023-01-31'
	AND tenure_paid_lens = 'N/A'
	AND d.date_of_last_view is not null 
)
SELECT  stream_type
        ,cte.adobe_tracking_id
       ,SUM(v.num_views_started)           AS total_content_starts
       ,SUM(v.num_seconds_played_no_ads)   AS total_watch_no_ads
       ,SUM(v.num_seconds_played_with_ads) AS total_watch_w_ads
	   ,
FROM cte
LEFT JOIN `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO` v
ON cte.adobe_tracking_id = v.adobe_tracking_id
where adobe_date <= '2023-01-31'
group by 1,2

Query is running:   0%|          |

""


In [2]:
%%bigquery --project nbcu-ds-sandbox-a-001
select stream_type, COUNT(distinct adobe_tracking_id) from `nbcu-ds-sandbox-a-001.SLi_sandbox.tenure_paid_lens_analysis` group by 1

Query is running:   0%|          |

Downloading:   0%|          |

,stream_type,f0_
0,trailer,930911
1,fullEpisodePlayer,26400
2,exclusiveChannel,34138
3,movie,14196
4,fullEventReplay,176
5,shortForm,4686
6,live,1077
7,vod,598
8,LIVE,46
9,shortform,36


In [3]:
%%bigquery --project nbcu-ds-sandbox-a-001
select stream_type, SUM(total_content_starts) from `nbcu-ds-sandbox-a-001.SLi_sandbox.tenure_paid_lens_analysis` group by 1

Query is running:   0%|          |

Downloading:   0%|          |

,stream_type,f0_
0,trailer,2306748
1,movie,0
2,exclusiveChannel,0
3,live,0
4,shortForm,0
5,fullEpisodePlayer,0
6,fullEventReplay,0
7,vod,0
8,shortform,0
9,LIVE,0


%%bigquery --project nbcu-ds-sandbox-a-001
select stream_type, SUM(total_watch_no_ads) from `nbcu-ds-sandbox-a-001.SLi_sandbox.tenure_paid_lens_analysis` group by 1

%%bigquery --project nbcu-ds-sandbox-a-001
select stream_type, SUM(total_watch_w_ads) from `nbcu-ds-sandbox-a-001.SLi_sandbox.tenure_paid_lens_analysis` group by 1

Analysis results:
- Tenure Paid Lens is N/A if there is not a gross_add_date
- gross_add_date <- monthly_active_account <- video_watched_trailing30 <- SILVER_VIDEO join with conditions `Silver_Video.num_views_started = 1 and (NOT(COALESCE(stream_type,"NULL") ='trailer' and COALESCE(lower(vdo_initiate),"NULL") like ('%auto%play%')))`
- N/A = Free users without activation event

Problems:
- tenure_paid_lens is not cumulative and is based on the first paying date. If a user joins 1/1, churns 1/31, rejoins 3/1, their tenure_paid_lens as of 3/31 would be 90 days instead of 60 days. Is this right?
- Should we be focusing on the last_paying_date instead of the first?
- Should we even be measuring free users, as tenure_paid_lens is focused on paying users only?

Solutions
- paid_tenure as cumulative paying days
- current_paid_tenure_streak as last_paying_date - current_date() 
- maa_tenure and current_active_streak to be inclusive of free users